In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
A = pd.read_csv("/content/drive/MyDrive/python/Deep Learning with Vaibhav Sir/Restaurant_Reviews.tsv",sep="\t")

In [ ]:
from nltk.stem import PorterStemmer
ps = PorterStemmer()
from nltk.stem import WordNetLemmatizer
wnl = WordNetLemmatizer()
import nltk
nltk.download('wordnet')


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
import re

In [ ]:
reviews = []
for i in A.Review:
    q1 = re.sub("[^a-zA-Z0-9 ]","",i)
    q2 = q1.upper()
    q3 = q2.split(" ")
    q4 = ""
    for j in q3:
        q4 = q4 + " " + str(wnl.lemmatize(ps.stem(j))).upper()
    reviews.append(q4)

In [ ]:
!pip install nltk

In [ ]:
reviews


[' WOW LOVE THI PLACE',
 ' CRUST IS NOT GOOD',
 ' NOT TASTI AND THE TEXTUR WA JUST NASTI',
 ' STOP BY DURE THE LATE MAY BANK HOLIDAY OFF RICK STEVE RECOMMEND AND LOVE IT',
 ' THE SELECT ON THE MENU WA GREAT AND SO WERE THE PRICE',
 ' NOW I AM GET ANGRI AND I WANT MY DAMN PHO',
 ' HONESLTI IT DIDNT TAST THAT FRESH',
 ' THE POTATO WERE LIKE RUBBER AND YOU COULD TELL THEY HAD BEEN MADE UP AHEAD OF TIME BE KEPT UNDER A WARMER',
 ' THE FRI WERE GREAT TOO',
 ' A GREAT TOUCH',
 ' SERVIC WA VERI PROMPT',
 ' WOULD NOT GO BACK',
 ' THE CASHIER HAD NO CARE WHAT SO EVER ON WHAT I HAD TO SAY IT STILL END UP BE WAYYY OVERPR',
 ' I TRI THE CAPE COD RAVOLI CHICKEN WITH CRANBERRYMMMM',
 ' I WA DISGUST BECAUS I WA PRETTI SURE THAT WA HUMAN HAIR',
 ' I WA SHOCK BECAUS NO SIGN INDIC CASH ONLI',
 ' HIGHLI RECOMMEND',
 ' WAITRESS WA A LITTL SLOW IN SERVIC',
 ' THI PLACE IS NOT WORTH YOUR TIME LET ALON VEGA',
 ' DID NOT LIKE AT ALL',
 ' THE BURRITTO BLAH',
 ' THE FOOD AMAZ',
 ' SERVIC IS ALSO CUTE',
 ' I COU

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
X = cv.fit_transform(reviews).toarray()

In [ ]:
Y = A[["Liked"]]

In [ ]:
#pd.DataFrame(cv.fit_transform(reviews).toarray(),columns=cv.get_feature_names())

In [ ]:
X.shape

(1000, 1728)

In [ ]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest=train_test_split(X,Y,test_size=0.2,random_state=21)

In [ ]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
model = gnb.fit(xtrain,ytrain)
pred = model.predict(xtest)
from sklearn.metrics import accuracy_score
accuracy_score(ytest,pred)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.705

In [ ]:
from keras.models import Sequential
from keras.layers import Dense,Dropout

nn = Sequential()
nn.add(Dense(500,input_dim=1728))
nn.add(Dense(500,activation="relu"))
nn.add(Dropout(0.15))
nn.add(Dense(500,activation="relu"))
nn.add(Dropout(0.15))
nn.add(Dense(500,activation="relu"))
nn.add(Dropout(0.15))
nn.add(Dense(1,activation="sigmoid"))

In [ ]:
from sys import maxsize
def modelbulder(hp):
  nn = Sequential()
  nn.add(Dense(500,input_dim= xtrain.shape[1]))
  # This is the parameters for Dense Layers
  x = hp.Int("units", min_value = 50,max_value = 500, step = 20)
  nn.add(Dense(x))
  y = hp.Choice("rate", values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7])
  nn.add(Dropout(rate=y))
  nn.add(Dense(1,activation = 'sigmoid'))
  nn.compile(loss="binary_crossentropy", metrics="accuracy")
  return nn

In [ ]:
!pip install keras-tuner

     |████████████████████████████████| 98 kB 3.6 MB/s 


In [ ]:
import keras_tuner as kr
tuner1 = kr.Hyperband(modelbulder, 
             objective="val_accuracy",
             max_epochs = 15,
             directory="/content/drive/MyDrive/Colab Notebooks/kerasTuner",
             project_name = "keras_tune1"
             )

In [ ]:
tuner1.search(xtrain,ytrain, epochs= 50, validation_split = 0.2)

Trial 30 Complete [00h 00m 07s]
val_accuracy: 0.8125

Best val_accuracy So Far: 0.84375
Total elapsed time: 00h 01m 58s
INFO:tensorflow:Oracle triggered exit


In [ ]:
tuner1.get_best_hyperparameters(num_trials=1)[0].get('units')

50

In [ ]:
tuner1.get_best_hyperparameters(num_trials=1)[0].get('rate')


0.3

In [ ]:
nn = Sequential()
nn.add(Dense(50,input_dim=1728))
nn.add(Dense(50,activation="relu"))
nn.add(Dropout(0.3))
nn.add(Dense(50,activation="relu"))
nn.add(Dropout(0.3))
nn.add(Dense(50,activation="relu"))
nn.add(Dropout(0.3))
nn.add(Dense(1,activation="sigmoid"))

In [ ]:
  nn.compile(loss="binary_crossentropy", metrics="accuracy")
  nn.fit(xtrain,ytrain,epochs = 100, validation_split=0.2)

Epoch 1/100
20/20 [==============================] - 1s 14ms/step - loss: 0.6944 - accuracy: 0.4922 - val_loss: 0.6933 - val_accuracy: 0.4688
Epoch 2/100
20/20 [==============================] - 0s 5ms/step - loss: 0.6803 - accuracy: 0.5859 - val_loss: 0.6826 - val_accuracy: 0.5063
Epoch 3/100
20/20 [==============================] - 0s 6ms/step - loss: 0.6376 - accuracy: 0.7047 - val_loss: 0.6258 - val_accuracy: 0.6875
Epoch 4/100
20/20 [==============================] - 0s 7ms/step - loss: 0.5031 - accuracy: 0.8203 - val_loss: 0.5225 - val_accuracy: 0.7875
Epoch 5/100
20/20 [==============================] - 0s 6ms/step - loss: 0.3191 - accuracy: 0.9016 - val_loss: 0.4598 - val_accuracy: 0.7625
Epoch 6/100
20/20 [==============================] - 0s 6ms/step - loss: 0.1995 - accuracy: 0.9375 - val_loss: 0.4621 - val_accuracy: 0.7812
Epoch 7/100
20/20 [==============================] - 0s 6ms/step - loss: 0.1296 - accuracy: 0.9688 - val_loss: 0.5647 - val_accuracy: 0.7563
Epoch 8/100


In [ ]:
pred = nn.predict(xtest> 0.5).astype("int32")
pred

array([[0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [0],
       [1],
       [0],
       [1],
       [1],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [1],
       [0],
       [0],
       [1],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [0],
       [0],
       [1],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [0],
       [1],
       [0],
       [0],
       [1],
       [1],
       [1],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [0],
       [0],
       [0],
       [1],
    

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(ytest,pred)

0.815